In [1]:
# === generate_sp500_pe.py ===

import pandas as pd
import numpy as np
import os

# === Paths ===
BASE_DIR = r"C:\Users\flass\OneDrive\AI Financial Model\S&P 500 Chatgpt Version"
df_price = pd.read_parquet(os.path.join(BASE_DIR, "df_labeled.parquet"))
df_fund = pd.read_parquet(os.path.join(BASE_DIR, "fundamentals.parquet"))

# === Filter Income and Balance Sheet Data ===
df_income = df_fund[df_fund["report_type"] == "Income_Statement"]
df_income = df_income[["ticker", "date", "netIncome"]].dropna()

df_balance = df_fund[df_fund["report_type"] == "Balance_Sheet"]
df_balance = df_balance[["ticker", "date", "commonStockSharesOutstanding"]].dropna()

# === Merge & Clean ===
df_fundamental = pd.merge(df_income, df_balance, on=["ticker", "date"], how="inner")
df_fundamental["date"] = pd.to_datetime(df_fundamental["date"])
df_fundamental["netIncome"] = pd.to_numeric(df_fundamental["netIncome"], errors="coerce")
df_fundamental["commonStockSharesOutstanding"] = pd.to_numeric(df_fundamental["commonStockSharesOutstanding"], errors="coerce")

df_fundamental = df_fundamental.sort_values(["ticker", "date"]).reset_index(drop=True)

# === Calculate TTM EPS ===
df_fundamental["ttm_net_income"] = (
    df_fundamental.groupby("ticker")["netIncome"]
    .rolling(4, min_periods=4).sum().reset_index(level=0, drop=True)
)

df_fundamental["ttm_shares"] = (
    df_fundamental.groupby("ticker")["commonStockSharesOutstanding"]
    .rolling(4, min_periods=4).mean().reset_index(level=0, drop=True)
)

df_fundamental["ttm_eps"] = df_fundamental["ttm_net_income"] / (df_fundamental["ttm_shares"] + 1e-6)
df_fundamental = df_fundamental.dropna(subset=["ttm_eps"])
df_fundamental = df_fundamental[df_fundamental["ttm_eps"] != 0]

# === Clean Price Data ===
df_price["date"] = pd.to_datetime(df_price["date"])
df_price = df_price.dropna(subset=["date", "ticker", "adjusted_close"])
df_price = df_price.sort_values(["ticker", "date"])
df_fundamental = df_fundamental.sort_values(["ticker", "date"])

# === Merge-asof TTM EPS to Daily Price ===
pe_dfs = []

for ticker in df_price["ticker"].unique():
    df_ticker_price = df_price[df_price["ticker"] == ticker]
    df_ticker_fund = df_fundamental[df_fundamental["ticker"] == ticker]

    if df_ticker_fund.empty:
        continue

    merged = pd.merge_asof(
        df_ticker_price,
        df_ticker_fund[["date", "ttm_eps"]],
        on="date",
        direction="backward"
    )

    merged["ticker"] = ticker
    merged["pe_ttm"] = merged["adjusted_close"] / (merged["ttm_eps"] + 1e-6)

    pe_dfs.append(merged)

# === Combine and Clean ===
df_pe = pd.concat(pe_dfs, ignore_index=True)
df_pe = df_pe.dropna(subset=["pe_ttm"])
df_pe["pe_ttm"] = pd.to_numeric(df_pe["pe_ttm"], errors="coerce")
df_pe = df_pe.replace([np.inf, -np.inf], np.nan).dropna(subset=["pe_ttm"])
df_pe["pe_ttm"] = df_pe["pe_ttm"].clip(lower=-500, upper=500).round(6)

# === Save Clean CSV ===
output_path = os.path.join(BASE_DIR, "sp500_pe_timeseries.csv")
df_pe[["ticker", "date", "adjusted_close", "ttm_eps", "pe_ttm"]].to_csv(output_path, index=False)

print(f"✅ PE time series saved to: {output_path}")
print(f"📊 Final shape: {df_pe.shape}")
print(f"🧼 Sample PE values:\n{df_pe[['pe_ttm']].head(10)}")


✅ PE time series saved to: C:\Users\flass\OneDrive\AI Financial Model\S&P 500 Chatgpt Version\sp500_pe_timeseries.csv
📊 Final shape: (3298667, 392)
🧼 Sample PE values:
      pe_ttm
0  23.435265
1  21.504510
2  23.435265
3  21.304746
4  21.870640
5  21.937258
6  22.436535
7  22.469843
8  22.869371
9  23.501794


In [7]:
df_out = pd.read_csv(output_path)
print(df_out.head(10))

  ticker        date  adjusted_close   ttm_eps     pe_ttm
0      A  1999-11-18         26.4545  1.128832  23.435286
1      A  1999-11-19         24.2750  1.128832  21.504529
2      A  1999-11-22         26.4545  1.128832  23.435286
3      A  1999-11-23         24.0495  1.128832  21.304765
4      A  1999-11-24         24.6883  1.128832  21.870660
5      A  1999-11-26         24.7635  1.128832  21.937277
6      A  1999-11-29         25.3271  1.128832  22.436555
7      A  1999-11-30         25.3647  1.128832  22.469863
8      A  1999-12-01         25.8157  1.128832  22.869391
9      A  1999-12-02         26.5296  1.128832  23.501815
